# Interfacing with Stellarium

Stellarium must be running on an internet-connected computer (mobile does not support this)

In [1]:
import datetime
import requests

In [2]:
api_url = "http://localhost:8090/api"

### Status

This is the return when no object is selected and the current time is active.
On my computer, the time does not update unless I switch between windows. Perhaps this wont be a problem when it is polled from another computer.

In [100]:
url = api_url + "/main/status?propId=56&actionId=33"
contents = requests.get(url).json()
contents

{'actionChanges': {'changes': {}, 'id': 33},
 'location': {'altitude': 69,
  'country': 'United States',
  'landscapeKey': '',
  'latitude': 33.22901153564453,
  'longitude': -87.57723236083984,
  'name': 'Northport',
  'planet': 'Earth',
  'role': 'N',
  'state': 'Alabama'},
 'propertyChanges': {'changes': {}, 'id': 56},
 'selectioninfo': '<h2>Polaris (Alrucaba - Cinosura - Cynosura - Tramontana - Yilduz - Mismar)<br />α UMi - 1 UMi - HIP 11767 - SAO 308 - HD 8890 - HR 424 - WDS J02318+8916Aa,Ab</h2>Type: <b>pulsating variable star, double star</b> (DCEPS)<br />Magnitude: <b>1.95</b> (reduced to <b>2.19</b> by <b>1.84</b> Airmasses)<br />Absolute Magnitude: -3.66<br />Color Index (B-V): <b>0.63</b><br />Magnitude range: <b>1.86</b>÷<b>2.13</b> (Photometric system: V)<br />RA/Dec (J2000.0):     2h31m51.61s/+89°15\'51.5"<br>RA/Dec (on date):     3h01m00.42s/+89°21\'42.1"<br>HA/Dec:    15h49m24.00s/+89°22\'33.8"  (apparent)<br>Az./Alt.: +0°37\'32.3"/+32°53\'27.0"  (apparent)<br>Gal. long

In [87]:
def printStatus(api_url):
    status = requests.get(api_url + "/main/status").json()
    print(status)

In [64]:
def getLocation(api_url):
    status = requests.get(api_url + "/main/status").json()
    return (status['location']['latitude'], status['location']['longitude'], status['location']['altitude'])

In [65]:
location = getLocation(api_url)
print(location)

(33.22901153564453, -87.57723236083984, 69)


In [66]:
def getUTC(api_url):
    status = requests.get(api_url + "/main/status").json()
    utc = datetime.datetime.fromisoformat(status['time']['utc'][:-1]) # must remove the Z on the end
    return (utc, status['time']['isTimeNow'])

In [67]:
utc = getUTC(api_url)
print(utc)

(datetime.datetime(2022, 10, 17, 22, 0, 9, 820000), True)


In [68]:
def getLST(api_url):
    utc = getUTC(api_url)[0]
    J0 = 367*utc.year - int(7*(utc.year + int((utc.month + 9)/12))/4) + int(275*utc.month/9) + utc.day + 1721013.5
    T0 = (J0 - 2451545)/36525
    GST0 = 100.4606184 + 36000.77004*T0 + 0.000387933*T0**2 - 2.583e-8*T0**3
    while GST0 > 360: GST0 -= 360
    UT = utc.hour + utc.minute/60 + utc.second/3600 + utc.microsecond/3600000000
    GST = GST0 + 360.98564724*UT/24
    while GST > 360: GST -= 360
    LST = (GST + longitude)/15
    LST_hour = int(LST)
    LST -= LST_hour
    LST *= 60
    LST_minute = int(LST)
    LST -= LST_minute
    LST *= 60
    LST_second = LST
    return (LST_hour, LST_minute, LST_second)

In [69]:
lst = getLST(api_url)
print(lst)

(17, 55, 23.86617636514643)


In [124]:
def selectTarget(api_url, target):
    status = requests.post(api_url + "/main/focus", data={'target': target})
    print(status.text)
    
selectTarget(api_url, 'Polaris')
printStatus(api_url)

true
{'location': {'altitude': 69, 'country': 'United States', 'landscapeKey': '', 'latitude': 33.22901153564453, 'longitude': -87.57723236083984, 'name': 'Northport', 'planet': 'Earth', 'role': 'N', 'state': 'Alabama'}, 'selectioninfo': '<h2>Polaris (Alrucaba - Cinosura - Cynosura - Tramontana - Yilduz - Mismar)<br />α UMi - 1 UMi - HIP 11767 - SAO 308 - HD 8890 - HR 424 - WDS J02318+8916Aa,Ab</h2>Type: <b>pulsating variable star, double star</b> (DCEPS)<br />Magnitude: <b>1.95</b> (reduced to <b>2.19</b> by <b>1.82</b> Airmasses)<br />Absolute Magnitude: -3.66<br />Color Index (B-V): <b>0.63</b><br />Magnitude range: <b>1.86</b>÷<b>2.13</b> (Photometric system: V)<br />RA/Dec (J2000.0):     2h31m51.61s/+89°15\'51.5"<br>RA/Dec (on date):     3h01m00.43s/+89°21\'42.1"<br>HA/Dec:    19h08m41.37s/+89°21\'17.6"  (apparent)<br>Az./Alt.: +0°44\'18.3"/+33°25\'02.3"  (apparent)<br>Gal. long./lat.: +123°16\'50.1"/+26°27\'41.9"<br>Supergal. long./lat.: +25°45\'15.6"/+15°24\'22.1"<br>Ecl. long./

In [125]:
def moveTelescopeToSelection(api_url):
    status = requests.post(api_url + "/stelaction/do", data={'id': 'actionMove_Telescope_To_Selection_1'})
    print(status.text)

moveTelescopeToSelection(api_url)

ok


In [123]:
def getInfo(api_url, obj):
    info = requests.get(api_url + "/objects/info", params={'name': obj, 'format': 'json'}).json()
    print(info)
    
getInfo(api_url, 'Polaris')

{'above-horizon': True, 'absolute-mag': -3.6631433963775635, 'airmass': 1.815628170967102, 'altitude': 33.41608428955078, 'altitude-geometric': 33.3908936266338, 'ambientInt': 0.027500001713633537, 'ambientLum': 0.0006828276091255248, 'appSidTm': '22h00m35.0s', 'azimuth': 0.7388060351048417, 'azimuth-geometric': 0.7388060351048417, 'bV': 0.6338582038879395, 'dec': 89.36170361584644, 'decJ2000': 89.26430756418556, 'distance-ly': 432.5681401818586, 'elat': 66.10637635368488, 'elatJ2000': 66.10149507011033, 'elong': 88.89054145513174, 'elongJ2000': 88.56818350808732, 'found': True, 'glat': 26.461631623407108, 'glong': 123.28058524715222, 'hourAngle-dd': 284.894211200574, 'hourAngle-hms': '18h59m34.6s', 'iauConstellation': 'UMi', 'localized-name': 'Polaris', 'meanSidTm': '22h00m35.8s', 'name': 'HIP 11767', 'parallax': 0.007540000000000001, 'period': 3.9696, 'ra': 45.251760701837945, 'raJ2000': 37.96503962425166, 'rise': '100h00m', 'rise-dhr': 100, 'set': '100h00m', 'set-dhr': 100, 'sglat':

In [116]:
info = requests.get(api_url + "/stelaction/list").json()
print(info)

{'Bright Novae': [{'id': 'actionShow_Novae_ConfigDialog', 'isCheckable': True, 'isChecked': False, 'text': 'Bright Novae configuration window'}], 'Date and Time': [{'id': 'actionNext_Transit', 'isCheckable': False, 'text': 'Next transit of the selected object'}, {'id': 'actionNext_Rising', 'isCheckable': False, 'text': 'Next rising of the selected object'}, {'id': 'actionNext_Setting', 'isCheckable': False, 'text': 'Next setting of the selected object'}, {'id': 'actionToday_Transit', 'isCheckable': False, 'text': "Today's transit of the selected object"}, {'id': 'actionToday_Rising', 'isCheckable': False, 'text': "Today's rising of the selected object"}, {'id': 'actionToday_Setting', 'isCheckable': False, 'text': "Today's setting of the selected object"}, {'id': 'actionPrevious_Transit', 'isCheckable': False, 'text': 'Previous transit of the selected object'}, {'id': 'actionPrevious_Rising', 'isCheckable': False, 'text': 'Previous rising of the selected object'}, {'id': 'actionPrevious

In [117]:
info = requests.get(api_url + "/stelproperty/list").json()
print(info)

{'AsterismMgr.asterismLineThickness': {'canNotify': True, 'isWritable': True, 'typeEnum': 2, 'typeString': 'int', 'value': 1, 'variantType': 'int'}, 'AsterismMgr.fontSize': {'canNotify': True, 'isWritable': True, 'typeEnum': 38, 'typeString': 'float', 'value': 14, 'variantType': 'float'}, 'AsterismMgr.linesColor': {'canNotify': True, 'isWritable': True, 'typeEnum': 1024, 'typeString': 'Vec3f', 'value': '[0.5, 0.5, 0.7]', 'variantType': 'Vec3f'}, 'AsterismMgr.linesDisplayed': {'canNotify': True, 'isWritable': True, 'typeEnum': 1, 'typeString': 'bool', 'value': False, 'variantType': 'bool'}, 'AsterismMgr.namesColor': {'canNotify': True, 'isWritable': True, 'typeEnum': 1024, 'typeString': 'Vec3f', 'value': '[0.5, 0.5, 0.7]', 'variantType': 'Vec3f'}, 'AsterismMgr.namesDisplayed': {'canNotify': True, 'isWritable': True, 'typeEnum': 1, 'typeString': 'bool', 'value': False, 'variantType': 'bool'}, 'AsterismMgr.rayHelperThickness': {'canNotify': True, 'isWritable': True, 'typeEnum': 2, 'typeSt